<a href="https://colab.research.google.com/github/Mabzak-Knight/penerjemah-variabel-masal/blob/main/Penerjemah_variabel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install googletrans==4.0.0-rc1
!pip install tqdm
!pip install json

from googletrans import Translator
from tqdm import tqdm

import os
import zipfile
import shutil

translator = Translator()

In [ ]:
def hapus_folder(nama_folder):
    try:
        shutil.rmtree(nama_folder)
        print(f"Folder '{nama_folder}' berhasil dihapus.")
    except Exception as e:
        print(f"Terjadi kesalahan saat menghapus folder '{nama_folder}': {e}")

In [ ]:
def ekstrak_zip(file_zip, direktori_ekstraksi):
    with zipfile.ZipFile(file_zip, 'r') as zip_ref:
        zip_ref.extractall(direktori_ekstraksi)

def zip_folder(sumber_folder, nama_file_zip):
    # Membuat objek ZipFile untuk menulis file zip
    with zipfile.ZipFile(nama_file_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(sumber_folder):
            for file in files:
                # Mendapatkan path lengkap dari file
                file_path = os.path.join(root, file)
                # Mendapatkan nama file relatif terhadap sumber_folder
                rel_path = os.path.relpath(file_path, sumber_folder)
                # Menambahkan file ke dalam zip
                zipf.write(file_path, arcname=rel_path)

In [ ]:
def terjemahkan_dan_simpan(nama_file_input, nama_file_output):
  # Membaca teks dari file
  with open(nama_file_input, 'r', encoding='utf-8') as file:
      hasil_baca = file.readlines()

  # Inisialisasi array untuk menyimpan hasil terjemahan
  terjemahan_array = []

  # Memproses setiap baris
  for baris in tqdm(hasil_baca, desc="Translating"):
      # Memisahkan teks bahasa Jepang dan bahasa Inggris
      try:
        bahasa_jepang, bahasa_inggris = baris.strip().split('=')
      except Exception as e:
        print(f"Skip Baris:{baris.strip}")
        terjemahan_array.append(baris.strip())
        continue



      # Menerjemahkan hanya bahasa Inggris
      try:
        terjemahan_inggris = translator.translate(bahasa_inggris, src='en', dest='id').text
      except Exception as e:
        print(f"\nBaris error: {bahasa_inggris}\nscript: {e}")
        terjemahan_inggris = bahasa_inggris



      # Menggabungkan kembali teks bahasa Jepang dan terjemahan bahasa Inggris
      hasil_terjemahan = f'{bahasa_jepang}={terjemahan_inggris}'

      # Menambahkan hasil terjemahan ke dalam array
      terjemahan_array.append(hasil_terjemahan)

  # Menyimpan terjemahan ke dalam file
  with open(nama_file_output, 'w', encoding='utf-8') as file:
    for terjemahan in terjemahan_array:
      file.write(terjemahan + '\n')

In [ ]:
def terjemahkan_folder(nama_folder, hasil_folder):
    if not os.path.exists(hasil_folder):
      os.makedirs(hasil_folder)
    for root, dirs, files in os.walk(nama_folder):
        for file in files:
            if file == 'translation.txt':
                # Membuat subfolder sesuai dengan struktur di nama_folder
                subfolder = os.path.relpath(root, nama_folder)
                output_folder = os.path.join(hasil_folder, subfolder)

                # Memastikan subfolder tujuan ada atau membuatnya jika belum ada
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                path_file = os.path.join(root, file)
                output_file = os.path.join(output_folder, 'translation.txt')  # Nama file output

                try:
                    terjemahkan_dan_simpan(path_file, output_file)
                except Exception as e:
                    print(f"Terjadi kesalahan pada file {path_file}: {e}")
                    continue  # Melanjutkan ke file berikutnya

In [ ]:
def menerjemahkan_zip(file_zip,hasil_zip):
  # Contoh pemanggilan fungsi
  hapus_folder('/content/mentah')
  hapus_folder('/content/terjemahan')

  # Contoh pemanggilan fungsi
  ekstrak_zip(file_zip, '/content/mentah')

  # Menerjemahkan file translte di dalam sub folder
  terjemahkan_folder('/content/mentah','/content/terjemahan')

  # MengZip File
  zip_folder('/content/terjemahan', hasil_zip)

In [ ]:
# Memanggil fungsi terjemahkan_dan_simpan
terjemahkan_dan_simpan('tes.txt', 'hasil_tes.txt')

In [ ]:
menerjemahkan_zip('/content/belum.zip','hasil_terjemahan.zip')

In [ ]:
  # Menerjemahkan file translte di dalam sub folder
  terjemahkan_folder('/content/mentah','/content/terjemahan')

In [ ]:
ekstrak_zip('/content/percobaan.zip', '/content/mentah')

In [ ]:
zip_folder('/content/terjemahan', 'hasil.zip')